# BULKI_Store Demo

## Starting Server

In [40]:
%env RUST_LOG=debug
print("Let's start")

env: RUST_LOG=debug
Let's start


In [41]:
# import subprocess

# # Start the server as a background process
# server_process = subprocess.Popen(
#     ['/opt/homebrew/bin/mpirun', '-np', '4', 'target/debug/bulkistore-server'],
#     env={'RUST_LOG': 'debug'},
#     stdout=subprocess.PIPE,
#     stderr=subprocess.PIPE
# )

# print("Server started with PID:", server_process.pid)

## Creating Arrays

In [88]:
import bkstore_client as bkc
import numpy as np
import time
from pprint import pprint


bkc.init()
dim_size = 12

print("5D array size = {} elements".format(dim_size**5))

# dim_size = 10
rd1 = np.random.rand(dim_size, dim_size, dim_size, dim_size, dim_size)
rd2 = np.random.rand(dim_size, dim_size, dim_size, dim_size, dim_size)

print("rd1.shape=", rd1.shape)
print("rd1 last 2 dim preview:")
print(rd1[0:1, 0:1, 0:1, 0:2, 0:])

print("rd2.shape=", rd2.shape)
print("rd2 last 2 dim preview:")
print(rd2[0:1, 0:1, 0:1, 0:2, 0:])

arr3 = np.random.rand(dim_size, dim_size, dim_size)
print("arr3.shape=", arr3.shape)
print("arr3 last 2 dim preview:")
print(arr3[0:1, 0:2, 0:])

5D array size = 248832 elements
rd1.shape= (12, 12, 12, 12, 12)
rd1 last 2 dim preview:
[[[[[0.63842598 0.96845306 0.01481329 0.97646342 0.26745643 0.55300935
     0.76250671 0.45296624 0.90932829 0.81259384 0.48157595 0.27124543]
    [0.23195725 0.50513194 0.88833803 0.53780968 0.86676205 0.84140611
     0.22164387 0.96742328 0.11598944 0.11122627 0.33168191 0.54487901]]]]]
rd2.shape= (12, 12, 12, 12, 12)
rd2 last 2 dim preview:
[[[[[0.32984777 0.17481162 0.25098855 0.84407873 0.55610243 0.82385136
     0.66002281 0.12241738 0.21949847 0.78241267 0.45293244 0.31712452]
    [0.13595862 0.46225032 0.86560736 0.20754165 0.00443883 0.78642073
     0.52194907 0.3114906  0.30751503 0.88976334 0.94329487 0.72550754]]]]]
arr3.shape= (12, 12, 12)
arr3 last 2 dim preview:
[[[0.50591856 0.14708923 0.19180399 0.97054161 0.0266377  0.65135754
   0.9802403  0.1674105  0.13207287 0.63438785 0.68358148 0.2209545 ]
  [0.86650443 0.38756928 0.12366659 0.36705942 0.86010541 0.90745961
   0.49397171 0.01

## Client-Side NDArray Arithmetic Operation

In [89]:
arr4 = bkc.polymorphic_add(rd1, rd2)

print("arr4.shape=", arr4.shape)
print("arr4 last 2 dim preview:")
print(arr4[0:1, 0:1, 0:1, 0:2, 0:])

arr4.shape= (12, 12, 12, 12, 12)
arr4 last 2 dim preview:
[[[[[0.96827375 1.14326468 0.26580184 1.82054214 0.82355886 1.37686071
     1.42252952 0.57538362 1.12882675 1.59500651 0.93450838 0.58836995]
    [0.36791587 0.96738226 1.75394539 0.74535133 0.87120089 1.62782683
     0.74359294 1.27891389 0.42350447 1.0009896  1.27497678 1.27038656]]]]]


## Remote Array Slicing

In [90]:
arr5 = bkc.array_slicing(
    arr4, [slice(0, 1), slice(0, 1), slice(0, 1), slice(0, 2), slice(0, 10, -2)]
)

print("arr5.shape=", arr5.shape)
print("arr5 last 2 dim preview:")
print(arr5[0:1, 0:1, 0:1, 0:2, 0:])

arr5.shape= (1, 1, 1, 2, 5)
arr5 last 2 dim preview:
[[[[[1.59500651 0.57538362 1.37686071 1.82054214 1.14326468]
    [1.0009896  1.27891389 1.62782683 0.74535133 0.96738226]]]]]


## Remote Array Arithmetic Operation

In [91]:
# for i in range(100000):
arr6 = bkc.times_two(rd1)

print("arr6.shape=", arr6.shape)
print("arr6:")
print(arr6)

arr6.shape= (12, 12, 12, 12, 12)
arr6:
[[[[[1.27685197e+00 1.93690612e+00 2.96265816e-02 ... 1.62518767e+00
     9.63151900e-01 5.42490868e-01]
    [4.63914499e-01 1.01026387e+00 1.77667606e+00 ... 2.22452538e-01
     6.63363813e-01 1.08975803e+00]
    [6.37602285e-01 6.44037637e-01 2.15792567e-01 ... 1.45293199e-01
     9.39952497e-01 1.95143680e+00]
    ...
    [1.75801202e+00 1.28882782e+00 1.92166693e+00 ... 2.02570903e-01
     4.45880904e-01 1.08284856e+00]
    [4.05386378e-01 3.99953372e-01 1.78874099e+00 ... 5.37230546e-01
     1.92980148e+00 1.61768037e+00]
    [2.67742356e-01 5.93853599e-03 1.36676867e+00 ... 1.34931078e+00
     7.18760212e-01 1.60227313e+00]]

   [[1.53897871e+00 1.81452247e+00 5.95239765e-01 ... 1.03173587e+00
     1.94399779e+00 1.08603362e+00]
    [1.35588714e-01 1.61554256e+00 4.97992896e-02 ... 4.81352379e-01
     6.23422647e-01 6.98935913e-01]
    [5.85618921e-01 1.36702368e+00 3.85668554e-01 ... 3.06205577e-01
     8.80609551e-01 1.53392437e+00]
    ..

## Creating Object with Multiple NDArrays

1. You can attach a series of metadata attributes directly to the object you are creating
2. You can attach a series of NDArrays directly to the object you are creating along with the metadata attributes of these NDArrays.
3. Each of these NDArrays will become a sub-object of the main object you are creating.

In [86]:
# Create an 3D array
# for _ in range(1000000):
name_id = str(np.random.rand(1)[0] * 1000000)[0:6]

gnn_arr1 = np.random.rand(100000, 2)
gnn_arr2 = np.random.rand(1, 100000)
dim_size = 18
rd1 = np.random.rand(dim_size, dim_size, dim_size, dim_size, dim_size)
rd2 = np.random.rand(dim_size, dim_size, dim_size, dim_size, dim_size)

obj_ids = bkc.create_objects(
    obj_name_key="name",
    parent_id=None,
    metadata={
        "name": "container{}".format(name_id),
        "type": "container",
        "keys": [
            "{}/arr6".format(name_id),
            "{}/arr5".format(name_id),
            "{}/arr4".format(name_id),
            "{}/arr3".format(name_id),
            "{}/gnn_arr1".format(name_id),
            "{}/gnn_arr2".format(name_id),
        ],
        "ranges": [(1, 100), (100, 200)],
        "part_num": 1,
        "part_size": 100,
    },
    data=arr3,
    array_meta_list=[
        {
            "name": "{}/arr6".format(name_id),
            "type": "array",
            "shape": arr6.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
        {
            "name": "{}/arr5".format(name_id),
            "type": "array",
            "shape": arr6.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
        {
            "name": "{}/arr4".format(name_id),
            "type": "array",
            "shape": arr4.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
        {
            "name": "{}/arr3".format(name_id),
            "type": "array",
            "shape": arr3.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
        {
            "name": "{}/gnn_arr1".format(name_id),
            "type": "array",
            "shape": gnn_arr1.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
        {
            "name": "{}/gnn_arr2".format(name_id),
            "type": "array",
            "shape": gnn_arr2.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
        {
            "name": "{}/rd1".format(name_id),
            "type": "array",
            "shape": rd1.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
        {
            "name": "{}/rd2".format(name_id),
            "type": "array",
            "shape": rd2.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
    ],
    array_data_list=[arr6, arr5, arr4, arr3, gnn_arr1, gnn_arr2, rd1, rd2],
)
print(obj_ids)

[32122341891919162988324184438177658, 32122341891919292115532704700006266, 32122341891922354275048944780541818, 32122341891922575635977833590128506, 32122341891925397987821115446493050, 32122341891925582455261856836976506, 32122341891927869851527001116344186, 32122341891928312573384774440550266, 32122341891933957277071333858312058]


In [47]:
print(rd2)

[[[[[8.57197151e-01 5.19644792e-01 8.47518337e-01 ... 9.35032411e-01
     9.04590675e-01 5.97112633e-01]
    [8.69485841e-01 8.77626911e-01 1.38795179e-01 ... 4.16992603e-01
     9.63948487e-01 9.54032603e-01]
    [1.29977426e-01 9.53534873e-01 6.76012612e-01 ... 4.02203227e-01
     9.29947397e-01 2.27507854e-02]
    ...
    [1.10668460e-01 7.44210507e-01 3.45239087e-01 ... 6.15342271e-01
     5.61682235e-01 3.31017950e-01]
    [5.56587504e-01 3.35764795e-01 8.48798189e-01 ... 9.98517501e-01
     9.05961715e-01 2.00266794e-01]
    [9.48168554e-01 6.78322853e-01 5.54094797e-01 ... 4.17370438e-01
     9.02275190e-01 4.85309861e-01]]

   [[7.73994168e-01 8.31142758e-01 2.52253134e-01 ... 3.86013902e-01
     7.73737372e-01 8.96473380e-01]
    [5.26616430e-01 6.58527747e-02 6.22331221e-01 ... 4.73711440e-01
     6.04135052e-01 2.98077581e-01]
    [1.02623849e-01 2.52113665e-01 2.06500006e-01 ... 4.08269073e-01
     5.74541798e-01 8.64730053e-01]
    ...
    [2.72327065e-01 1.05294668e-01 3.

## Retrieve Metadata

### Retrieve a single metadata attributes from a single object


In HydraGNN, we need to retrieve a series of metadata attributes from a container. 

If every single metadata retrieval requires a single request, that would be too much overhead.

MetaPS is a new metric we designed to calculate the number of metadata attributes we can retrieve from one object within a second.


In [48]:
meta_keys = ["type", "keys", "ranges", "name", "part_num", "part_size", "title"]

start = time.time()
for i in range(1000):
    k = i % len(meta_keys)
    result = bkc.get_object_metadata(
        obj_ids[0],
        meta_keys=[meta_keys[k]],  # retreiving SINGLE!!!
    )
    if i < len(meta_keys):
        print(result)

print(
    "Single Metadata retrieval of single object : TPS = {} MetaPS: {}".format(
        1000 / (time.time() - start), 1000 / (time.time() - start)
    )
)

{'obj_id': 32121173564509879056484205307490913, 'metadata': {'type': 'container'}, 'sub_obj_metadata': None}
{'obj_id': 32121173564509879056484205307490913, 'metadata': {'keys': ['354736/arr6', '354736/arr5', '354736/arr4', '354736/arr3', '354736/gnn_arr1', '354736/gnn_arr2']}, 'sub_obj_metadata': None}
{'obj_id': 32121173564509879056484205307490913, 'metadata': {'ranges': [(1, 100), (100, 200)]}, 'sub_obj_metadata': None}
{'obj_id': 32121173564509879056484205307490913, 'metadata': {'name': 'container354736'}, 'sub_obj_metadata': None}
{'obj_id': 32121173564509879056484205307490913, 'metadata': {'part_num': 1}, 'sub_obj_metadata': None}
{'obj_id': 32121173564509879056484205307490913, 'metadata': {'part_size': 100}, 'sub_obj_metadata': None}
{'obj_id': 32121173564509879056484205307490913, 'metadata': {}, 'sub_obj_metadata': None}
Single Metadata retrieval of single object : TPS = 1353.3431186006014 MetaPS: 1353.3391885586918


In [74]:
meta_keys = ["type", "keys", "ranges", "name", "part_num", "part_size", "title"]

start = time.time()
for i in range(1000):
    k = i % len(meta_keys)
    result = bkc.get_object_metadata(
        obj_id="container{}".format(name_id),
        meta_keys=[meta_keys[k]],  # retreiving SINGLE!!!
    )
    if i < len(meta_keys):
        print(result)

print(
    "Single Metadata retrieval of single object : TPS = {} MetaPS: {}".format(
        1000 / (time.time() - start), 1000 / (time.time() - start)
    )
)

{'obj_id': 32121816015014274547986495604910246, 'metadata': {'type': 'container'}, 'sub_obj_metadata': None}
{'obj_id': 32121816015014274547986495604910246, 'metadata': {'keys': ['27866./arr6', '27866./arr5', '27866./arr4', '27866./arr3', '27866./gnn_arr1', '27866./gnn_arr2']}, 'sub_obj_metadata': None}
{'obj_id': 32121816015014274547986495604910246, 'metadata': {'ranges': [(1, 100), (100, 200)]}, 'sub_obj_metadata': None}
{'obj_id': 32121816015014274547986495604910246, 'metadata': {'name': 'container27866.'}, 'sub_obj_metadata': None}
{'obj_id': 32121816015014274547986495604910246, 'metadata': {'part_num': 1}, 'sub_obj_metadata': None}
{'obj_id': 32121816015014274547986495604910246, 'metadata': {'part_size': 100}, 'sub_obj_metadata': None}
{'obj_id': 32121816015014274547986495604910246, 'metadata': {}, 'sub_obj_metadata': None}
Single Metadata retrieval of single object : TPS = 1104.8271705619934 MetaPS: 1104.826006467268


### Retrieving multiple metadata attributes from a single object

Now we can combine the metadata retrieval of a single container into one single request. 

MetaPS is a new metric we designed to calculate the number of metadata attributes we can retrieve from one object within a second.

Slight drop on TPS but MEGA IMPROVEMENT for MetaPS

In [50]:
meta_keys = ["type", "keys", "ranges", "name", "part_num", "part_size"]

start = time.time()
for i in range(1000):
    result = bkc.get_object_metadata(
        obj_ids[0],
        meta_keys=meta_keys,  # retrieving MULTIPLE!!!
    )
pprint(result)
print(
    "\n Multiple Metadata retrieval of single object : TPS={}, MetaPS: {} ".format(
        1000 / (time.time() - start), 1000 * len(meta_keys) / (time.time() - start)
    )
)

{'metadata': {'keys': ['354736/arr6',
                       '354736/arr5',
                       '354736/arr4',
                       '354736/arr3',
                       '354736/gnn_arr1',
                       '354736/gnn_arr2'],
              'name': 'container354736',
              'part_num': 1,
              'part_size': 100,
              'ranges': [(1, 100), (100, 200)],
              'type': 'container'},
 'obj_id': 32121173564509879056484205307490913,
 'sub_obj_metadata': None}

 Multiple Metadata retrieval of single object : TPS=1309.2977752915895, MetaPS: 7855.776842683582 


### Retrieving single attributes from a single sub-object

In [51]:
meta_keys = ["name", "type", "shape", "vcount", "voffset", "vdim"]

start = time.time()
for i in range(1000):
    k = i % len(meta_keys)
    result = bkc.get_object_metadata(
        obj_id=obj_ids[1],
        meta_keys=[meta_keys[k]],  # retreiving SINGLE!!!
    )
    if i < len(meta_keys):
        print(result)

print(
    "Single Metadata retrieval of a single sub-object : TPS={}, MetaPS: {}".format(
        1000 / (time.time() - start), 1000 / (time.time() - start)
    )
)

{'obj_id': 32121173564509989736948651859767905, 'metadata': {'name': '354736/arr6'}, 'sub_obj_metadata': None}
{'obj_id': 32121173564509989736948651859767905, 'metadata': {'type': 'array'}, 'sub_obj_metadata': None}
{'obj_id': 32121173564509989736948651859767905, 'metadata': {'shape': [12, 12, 12, 12, 12]}, 'sub_obj_metadata': None}
{'obj_id': 32121173564509989736948651859767905, 'metadata': {'vcount': 100}, 'sub_obj_metadata': None}
{'obj_id': 32121173564509989736948651859767905, 'metadata': {'voffset': 0}, 'sub_obj_metadata': None}
{'obj_id': 32121173564509989736948651859767905, 'metadata': {'vdim': 0}, 'sub_obj_metadata': None}
Single Metadata retrieval of a single sub-object : TPS=1474.104179348292, MetaPS: 1474.102625111279


### Retrieving multiple attributes from single sub-objects

In [52]:
meta_keys = ["name", "type", "shape", "vcount", "voffset", "vdim"]

start = time.time()
for i in range(1000):
    result = bkc.get_object_metadata(
        obj_ids[1],
        meta_keys=meta_keys,  # retreiving SINGLE!!!
    )

pprint(result)
print(
    "\n Multiple Metadata retrieval of a single sub-object : TPS={}, MetaPS: {}".format(
        1000 / (time.time() - start), 1000 * len(meta_keys) / (time.time() - start)
    )
)

{'metadata': {'name': '354736/arr6',
              'shape': [12, 12, 12, 12, 12],
              'type': 'array',
              'vcount': 100,
              'vdim': 0,
              'voffset': 0},
 'obj_id': 32121173564509989736948651859767905,
 'sub_obj_metadata': None}

 Multiple Metadata retrieval of a single sub-object : TPS=1393.3032613357136, MetaPS: 8359.808459855796


### Retrieving multiple attributes from multiple sub-objects

In [53]:
meta_keys = ["name", "type", "shape", "vcount", "voffset", "vdim"]

start = time.time()
for i in range(1000):
    result = bkc.get_object_metadata(
        obj_ids[0],
        sub_meta_keys=meta_keys,  # retreiving MULTIPLE for every sub-object!!!
    )

pprint(result)
print(
    "Multiple Metadata retrieval of a single sub-object : TPS={}, MetaPS: {}".format(
        1000 / (time.time() - start), 1000 * len(meta_keys) * 3 / (time.time() - start)
    )
)

{'metadata': {},
 'obj_id': 32121173564509879056484205307490913,
 'sub_obj_metadata': [{'metadata': {'name': '354736/rd1',
                                    'shape': [18, 18, 18, 18, 18],
                                    'type': 'array',
                                    'vcount': 100,
                                    'vdim': 0,
                                    'voffset': 0},
                       'name': '354736/rd1',
                       'obj_id': 32121173564520449040838470945337953},
                      {'metadata': {'name': '354736/rd2',
                                    'shape': [18, 18, 18, 18, 18],
                                    'type': 'array',
                                    'vcount': 100,
                                    'vdim': 0,
                                    'voffset': 0},
                       'name': '354736/rd2',
                       'obj_id': 32121173564537881213988130766582369},
                      {'metadata': {'name': '3547

### Retrieving different set of attributes from multiple sub-objects

In [54]:
# meta_keys=["name",
#             "type",
#             "shape",
#             "vcount",
#             "voffset",
#             "vdim"]

start = time.time()
for i in range(1000):
    result = bkc.get_object_metadata(
        obj_ids[0],
        sub_meta_keys={
            "{}/arr5".format(name_id): ["name", "type", "voffset"],
            "{}/arr4".format(name_id): ["type", "vcount"],
        },  # retrieving different set of metadata for different sub-objects
    )

pprint(result)
print(
    "\n Multiple Metadata retrieval of a single sub-object : TPS={}, MetaPS: {}".format(
        1000 / (time.time() - start), 1000 * 5 / (time.time() - start)
    )
)

{'metadata': {},
 'obj_id': 32121173564509879056484205307490913,
 'sub_obj_metadata': [{'metadata': {'name': '354736/arr5',
                                    'type': 'array',
                                    'voffset': 0},
                       'name': '354736/arr5',
                       'obj_id': 32121173564513051896464891940303457},
                      {'metadata': {'type': 'array', 'vcount': 100},
                       'name': '354736/arr4',
                       'obj_id': 32121173564513217917161559621235297}]}

 Multiple Metadata retrieval of a single sub-object : TPS=1195.7947795395355, MetaPS: 5978.967079290981


### Retrieving array of a single object

In [55]:
start = time.time()
for i in range(1000):
    result = bkc.get_object_data(
        obj_ids[0],
        region=[slice(0, 1), slice(0, 1), slice(0, 2, -1)],
    )
pprint(result)
print(
    "\n retrieving array slice of a single 3D array, TPS={}, Sample/s = {}".format(
        1000 / (time.time() - start), (1000 / 6) / (time.time() - start)
    )
)

{'array_slice': array([[[0.24207276, 0.51412727]]]), 'sub_obj_slices': []}

 retrieving array slice of a single 3D array, TPS=1127.9986682297217, Sample/s = 187.99952523923602


In [56]:
start = time.time()
for i in range(1000):
    result = bkc.get_object_data(
        obj_ids[0],
        region=[
            slice(2, 4, None),
            slice(None),
            slice(None),
        ],
    )
pprint(result)
print(
    "\n retrieving array slice of a single 3D array, TPS={}, Sample/s = {}".format(
        1000 / (time.time() - start), (1000 / 6) / (time.time() - start)
    )
)

{'array_slice': array([[[9.18338155e-01, 2.24838086e-01, 4.52147708e-01, 5.03717650e-01,
         3.89137439e-01, 1.43900647e-01, 8.07484417e-01, 6.54510243e-01,
         1.30673977e-01, 2.76842835e-01, 1.66447647e-02, 3.71537097e-01],
        [9.98478653e-01, 5.34040746e-02, 5.13279185e-01, 6.52789267e-02,
         3.25063397e-01, 7.38238278e-01, 1.12935644e-01, 2.63705988e-01,
         3.63702400e-01, 7.04698535e-02, 1.58790867e-01, 8.82802699e-01],
        [4.86923897e-01, 8.59144990e-01, 9.97188386e-01, 9.87627230e-01,
         5.89193386e-01, 1.72595707e-01, 4.49882362e-01, 8.56619141e-01,
         3.25354719e-01, 7.69616759e-02, 1.81125836e-01, 6.76471718e-01],
        [7.69673783e-01, 1.79419884e-01, 3.53849456e-01, 8.47632944e-02,
         8.48670979e-01, 4.38913581e-01, 5.76214859e-01, 4.71843608e-01,
         2.94480115e-01, 6.76002969e-01, 7.23168296e-01, 1.23425862e-01],
        [9.96636819e-01, 3.33295632e-01, 8.26307859e-02, 9.38953749e-01,
         8.15257116e-01, 3.3716

### Retrieving array slices of multiple sub-objects (Large subarray included)

In [57]:
start = time.time()
for i in range(1000):
    result = bkc.get_object_data(
        obj_ids[0],
        region=[slice(0, 1), slice(0, 2), slice(0, 2, -1)],
        sub_obj_regions=[
            (
                "{}/arr5".format(name_id),
                [slice(0, 1), slice(0, 1), slice(0, 1), slice(0, 2), slice(0, 2, -1)],
            ),
            (
                "{}/arr4".format(name_id),
                [
                    slice(0, 2, -1),
                    slice(0, 1),
                    slice(0, 1),
                    slice(0, 1),
                    slice(0, 2, -1),
                ],
            ),
            (
                "{}/arr3".format(name_id),
                [slice(0, 1), slice(0, 2), slice(0, 10, -2)],
            ),
            (
                "{}/arr6".format(name_id),
                [slice(0, 1), slice(0, 1), slice(0, 1), slice(0, 2), slice(0, 2, -1)],
            ),
        ],
    )
pprint(result)
print(
    "\n retrieving array slice of a single 3D array, TPS={}, Sample/s={}".format(
        1000 / (time.time() - start), (1000 / (4 / 6)) / (time.time() - start)
    )
)

{'array_slice': array([[[0.24207276, 0.51412727],
        [0.78121905, 0.12242931]]]),
 'sub_obj_slices': [{'array': array([[[[[0.98767158, 1.10883077],
          [0.31152137, 0.99121944]]]]]),
                     'id': 32121173564513051896464891940303457,
                     'name': '354736/arr5'},
                    {'array': array([[[[[0.12167471, 0.64709467]]]],



       [[[[1.23376175, 0.47026345]]]]]),
                     'id': 32121173564513217917161559621235297,
                     'name': '354736/arr4'},
                    {'array': array([[[0.75436391, 0.28890424, 0.1618159 , 0.06812954, 0.24207276],
        [0.8574423 , 0.46612193, 0.12364062, 0.29296782, 0.78121905]]]),
                     'id': 32121173564515892695052251801186913,
                     'name': '354736/arr3'},
                    {'array': array([[[[[0.8383082 , 0.79155813],
          [0.79358011, 0.24723585]]]]]),
                     'id': 32121173564509989736948651859767905,
                     '

### Retrieving slices of multiple objects (only GNN arrays)

In [58]:
start = time.time()
for i in range(1000):
    result = bkc.get_object_data(
        obj_ids[0],
        region=None,
        sub_obj_regions=[
            (
                "{}/gnn_arr1".format(name_id),
                [slice(0, 20), slice(0, 1)],
            ),
            (
                "{}/gnn_arr1".format(name_id),
                [
                    slice(0, 10, -1),
                    slice(0, 1),
                ],
            ),
            (
                "{}/gnn_arr1".format(name_id),
                [
                    slice(0, 15, -1),
                    slice(0, 2),
                ],
            ),
            (
                "{}/gnn_arr2".format(name_id),
                [
                    slice(0, 1),
                    slice(0, 20, -1),
                ],
            ),
            (
                "{}/gnn_arr2".format(name_id),
                [
                    slice(0, 1),
                    slice(0, 20, -1),
                ],
            ),
            (
                "{}/gnn_arr2".format(name_id),
                [
                    slice(0, 1),
                    slice(0, 20, -1),
                ],
            ),
        ],
    )
pprint(result)
print(
    "\n retrieving array slice of a single 3D array, TPS={}, Sample/s={}".format(
        1000 / (time.time() - start), (1000) / (time.time() - start)
    )
)

{'array_slice': None,
 'sub_obj_slices': [{'array': array([[0.17459538],
       [0.97292865],
       [0.82389662],
       [0.88051344],
       [0.93704434],
       [0.26758978],
       [0.35406637],
       [0.91982179],
       [0.06214039],
       [0.28411887],
       [0.20110416],
       [0.32489322],
       [0.43549561],
       [0.13785823],
       [0.22757067],
       [0.93108772],
       [0.77264823],
       [0.58212372],
       [0.18603117],
       [0.72169339]]),
                     'id': 32121173564516021822260772063015521,
                     'name': '354736/gnn_arr1'},
                    {'array': array([[0.28411887],
       [0.06214039],
       [0.91982179],
       [0.35406637],
       [0.26758978],
       [0.93704434],
       [0.88051344],
       [0.82389662],
       [0.97292865],
       [0.17459538]]),
                     'id': 32121173564516021822260772063015521,
                     'name': '354736/gnn_arr1'},
                    {'array': array([[0.22757067, 0.501430

In [95]:
meta_keys = ["type", "keys", "ranges", "name", "part_num", "part_size", "title"]
# Create an 3D array
for _ in range(1):
    name_id = str(np.random.rand(1)[0] * 1000000)[0:6]

    gnn_arr1 = np.random.rand(100000, 2)
    gnn_arr2 = np.random.rand(1, 100000)
    dim_size = 12
    rd1 = np.random.rand(dim_size, dim_size, dim_size, dim_size, dim_size)
    rd2 = np.random.rand(dim_size, dim_size, dim_size, dim_size, dim_size)

    obj_ids = bkc.create_objects(
        obj_name_key="name",
        parent_id=None,
        metadata={
            "name": f"trainset/part_{name_id}",
            "type": "container",
            "keys": [
                "{}/arr6".format(name_id),
                "{}/arr5".format(name_id),
                "{}/arr4".format(name_id),
                "{}/arr3".format(name_id),
                "{}/gnn_arr1".format(name_id),
                "{}/gnn_arr2".format(name_id),
            ],
            "ranges": [(1, 100), (100, 200)],
            "range": (1, 100),
            "part_num": 1,
            "part_size": 100,
        },
        data=arr3,
        array_meta_list=[
            {
                "name": "{}/arr6".format(name_id),
                "type": "array",
                "shape": arr6.shape,
                "vcount": 100,
                "voffset": 0,
                "vdim": 0,
            },
            {
                "name": "{}/arr5".format(name_id),
                "type": "array",
                "shape": arr6.shape,
                "vcount": 100,
                "voffset": 0,
                "vdim": 0,
            },
            {
                "name": "{}/arr4".format(name_id),
                "type": "array",
                "shape": arr4.shape,
                "vcount": 100,
                "voffset": 0,
                "vdim": 0,
            },
            {
                "name": "{}/arr3".format(name_id),
                "type": "array",
                "shape": arr3.shape,
                "vcount": 100,
                "voffset": 0,
                "vdim": 0,
            },
            {
                "name": "{}/gnn_arr1".format(name_id),
                "type": "array",
                "shape": gnn_arr1.shape,
                "vcount": 100,
                "voffset": 0,
                "vdim": 0,
            },
            {
                "name": "{}/gnn_arr2".format(name_id),
                "type": "array",
                "shape": gnn_arr2.shape,
                "vcount": 100,
                "voffset": 0,
                "vdim": 0,
            },
            {
                "name": "{}/rd1".format(name_id),
                "type": "array",
                "shape": rd1.shape,
                "vcount": 100,
                "voffset": 0,
                "vdim": 0,
            },
            {
                "name": "{}/rd2".format(name_id),
                "type": "array",
                "shape": rd2.shape,
                "vcount": 100,
                "voffset": 0,
                "vdim": 0,
            },
        ],
        array_data_list=[arr6, arr5, arr4, arr3, gnn_arr1, gnn_arr2, rd1, rd2],
    )
    result = bkc.get_object_metadata(
        obj_id=f"trainset/part_{name_id}",
        meta_keys=meta_keys,  # retreiving SINGLE!!!
    )
    pprint(result)

{'metadata': {'keys': ['823171/arr6',
                       '823171/arr5',
                       '823171/arr4',
                       '823171/arr3',
                       '823171/gnn_arr1',
                       '823171/gnn_arr2'],
              'name': 'trainset/part_823171',
              'part_num': 1,
              'part_size': 100,
              'ranges': [(1, 100), (100, 200)],
              'type': 'container'},
 'obj_id': 32122385356498091075690509862679745,
 'sub_obj_metadata': None}


## Closing the Store

In [59]:
# server_process.terminate()
# server_process.wait()
# print("Server process terminated.")